In [12]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
        df['prefixo'] = df['Papel'].astype(str).str[:4]
        df=df.drop_duplicates(subset='prefixo', keep='first')
        # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [13]:
# A empresa e Indicadores fundamentalistas dados do statusinvest.com.br
#(pode ter atraso, dados não são pegos diretamente no SI)

url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
funds = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )

In [14]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [15]:
fundsSI =  funds[funds[' LIQUIDEZ MEDIA DIARIA'] > 250000] 
fundsSI =  fundsSI[(fundsSI[' LPA'] > 0) & (fundsSI[' VPA'] > 0) ]
# fundsSI

In [16]:
fundsSI['valor_intrinseco'] = np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA'])
fundsSI['Delta'] = fundsSI['valor_intrinseco'] / fundsSI['PRECO'] -1
fundsSI["Rank"]   = fundsSI['Delta'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA', 'P/L' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA,P/L
1,VVEO3,2.42,15.209882,204.0,5.285075,1.01,10.18,2.40
2,HBOR3,2.05,9.502000,203.0,3.635122,0.38,10.56,5.33
3,HBRE3,5.08,22.461868,202.0,3.421627,1.14,19.67,4.47
4,SHOW3,1.54,6.105755,201.0,2.964776,0.63,2.63,2.46
5,ETER3,5.90,21.034793,200.0,2.565219,1.71,11.50,3.46
6,MTRE3,3.58,11.970620,199.0,2.343749,0.69,9.23,5.22
7,EUCA4,15.22,50.730169,198.0,2.333125,4.30,26.60,3.54
8,COCE5,29.40,97.739130,197.0,2.324460,7.65,55.50,3.84
9,ALLD3,7.60,25.199554,196.0,2.315731,1.67,16.90,4.60
10,BRSR6,11.36,36.772279,195.0,2.236996,2.46,24.43,4.62


In [17]:
fundsSI =  fundsSI[(fundsSI['P/L'] > 0)]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,VVEO3,2.42,15.209882,204.0,5.285075,1.01,10.18
2,HBOR3,2.05,9.502000,203.0,3.635122,0.38,10.56
3,HBRE3,5.08,22.461868,202.0,3.421627,1.14,19.67
4,SHOW3,1.54,6.105755,201.0,2.964776,0.63,2.63
5,ETER3,5.90,21.034793,200.0,2.565219,1.71,11.50
6,MTRE3,3.58,11.970620,199.0,2.343749,0.69,9.23
7,EUCA4,15.22,50.730169,198.0,2.333125,4.30,26.60
8,COCE5,29.40,97.739130,197.0,2.324460,7.65,55.50
9,ALLD3,7.60,25.199554,196.0,2.315731,1.67,16.90
10,BRSR6,11.36,36.772279,195.0,2.236996,2.46,24.43


In [18]:
fundsSI =  fundsSI[(fundsSI['CAGR RECEITAS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
2,HBOR3,2.05,9.502000,203.0,3.635122,0.38,10.56
3,HBRE3,5.08,22.461868,202.0,3.421627,1.14,19.67
4,SHOW3,1.54,6.105755,201.0,2.964776,0.63,2.63
5,ETER3,5.90,21.034793,200.0,2.565219,1.71,11.50
6,MTRE3,3.58,11.970620,199.0,2.343749,0.69,9.23
7,EUCA4,15.22,50.730169,198.0,2.333125,4.30,26.60
8,COCE5,29.40,97.739130,197.0,2.324460,7.65,55.50
9,ALLD3,7.60,25.199554,196.0,2.315731,1.67,16.90
10,BRSR6,11.36,36.772279,195.0,2.236996,2.46,24.43
11,SOMA3,5.93,18.293851,194.0,2.084967,2.01,7.40


In [19]:
fundsSI =  fundsSI[(fundsSI['CAGR LUCROS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
4,SHOW3,1.54,6.105755,201.0,2.964776,0.63,2.63
6,MTRE3,3.58,11.970620,199.0,2.343749,0.69,9.23
7,EUCA4,15.22,50.730169,198.0,2.333125,4.30,26.60
8,COCE5,29.40,97.739130,197.0,2.324460,7.65,55.50
9,ALLD3,7.60,25.199554,196.0,2.315731,1.67,16.90
10,BRSR6,11.36,36.772279,195.0,2.236996,2.46,24.43
11,SOMA3,5.93,18.293851,194.0,2.084967,2.01,7.40
14,JHSF3,3.96,11.172735,191.0,1.821398,0.76,7.30
15,ALOS3,20.80,58.674950,190.0,1.820911,5.97,25.63
16,BMGB4,3.38,9.531317,189.0,1.819916,0.56,7.21


In [20]:
fundsSI =  fundsSI[(fundsSI['DIVIDA LIQUIDA / EBIT'] <4) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
4,SHOW3,1.54,6.105755,201.0,2.964776,0.63,2.63
7,EUCA4,15.22,50.730169,198.0,2.333125,4.30,26.60
8,COCE5,29.40,97.739130,197.0,2.324460,7.65,55.50
9,ALLD3,7.60,25.199554,196.0,2.315731,1.67,16.90
11,SOMA3,5.93,18.293851,194.0,2.084967,2.01,7.40
14,JHSF3,3.96,11.172735,191.0,1.821398,0.76,7.30
15,ALOS3,20.80,58.674950,190.0,1.820911,5.97,25.63
17,GOAU4,10.79,28.314025,188.0,1.624099,2.08,17.13
18,GOAU3,10.80,28.314025,187.0,1.621669,2.08,17.13
20,VLID3,15.35,39.507847,185.0,1.573801,3.69,18.80


In [21]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
df_limpo[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
4,SHOW3,1.54,6.105755,201.0,2.964776,0.63,2.63
7,EUCA4,15.22,50.730169,198.0,2.333125,4.30,26.60
8,COCE5,29.40,97.739130,197.0,2.324460,7.65,55.50
9,ALLD3,7.60,25.199554,196.0,2.315731,1.67,16.90
11,SOMA3,5.93,18.293851,194.0,2.084967,2.01,7.40
14,JHSF3,3.96,11.172735,191.0,1.821398,0.76,7.30
15,ALOS3,20.80,58.674950,190.0,1.820911,5.97,25.63
17,GOAU4,10.79,28.314025,188.0,1.624099,2.08,17.13
20,VLID3,15.35,39.507847,185.0,1.573801,3.69,18.80
21,NEOE3,18.26,46.570495,184.0,1.550410,3.97,24.28


In [22]:
# linhas_com_nan = top20[top20.isna().any(axis=1)]
# linhas_com_nan

### Porque 22,5?

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é, na verdade, o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce.

Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos.

O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores.

Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

No próximo tópico, abordaremos a calculadora do preço justo usando a Fórmula de Graham.